In [11]:
from datasets import Dataset
from collections import Counter
from dotenv import load_dotenv
load_dotenv()
from config import Config
config = Config()
import os
import glob
import plotly.graph_objects as go



In [56]:
import pandas as pd
for model in glob.glob('preds/pred*prompt1'):
    fig=go.Figure()
    ds = Dataset.from_file(f"{model}/data-00000-of-00001.arrow")
    df_pandas = ds.to_pandas()
    complete_list = []
    for lang in config.languages:
        fig=go.Figure()
        group = df_pandas.groupby(f'{lang}: Pred label')[f'{lang}: Pred label'].count().to_frame('count').reset_index()

        missing = [x for x in [-1,1,0] if x not in group[f'{lang}: Pred label'].unique()]
        if len(missing) > 0:
            for i in range(len(missing)):
                group.at[group.index[-1]+1+i, f'{lang}: Pred label'] = missing[i]
                group.at[group.index[-1]+1+i, 'count'] = 0


        complete_list.append((lang, group[group[f'{lang}: Pred label'] == -1]['count'].iloc[0], group[group[f'{lang}: Pred label'] == 0]['count'].iloc[0], group[group[f'{lang}: Pred label'] == 1]['count'].iloc[0]))
    
    df = pd.DataFrame(complete_list, columns =['Language', '-1', '0', '1'])
    fig = go.Figure(data=[
    go.Bar(name='0', x=df['Language'], y=df['0']),
    go.Bar(name='1', x=df['Language'], y=df['1']),
    go.Bar(name='NaN', x=df['Language'], y=df['-1'])
    ])
    fig.update_layout(title=model)
    fig.show()

In [57]:
for model in glob.glob('preds/pred*prompt2'):
    fig=go.Figure()
    ds = Dataset.from_file(f"{model}/data-00000-of-00001.arrow")
    df_pandas = ds.to_pandas()
    complete_list = []
    for lang in config.languages:
        fig=go.Figure()
        group = df_pandas.groupby(f'{lang}: Pred label')[f'{lang}: Pred label'].count().to_frame('count').reset_index()

        missing = [x for x in [-1,1,0] if x not in group[f'{lang}: Pred label'].unique()]
        if len(missing) > 0:
            for i in range(len(missing)):
                group.at[group.index[-1]+1+i, f'{lang}: Pred label'] = missing[i]
                group.at[group.index[-1]+1+i, 'count'] = 0


        complete_list.append((lang, group[group[f'{lang}: Pred label'] == -1]['count'].iloc[0], group[group[f'{lang}: Pred label'] == 0]['count'].iloc[0], group[group[f'{lang}: Pred label'] == 1]['count'].iloc[0]))
    
    df = pd.DataFrame(complete_list, columns =['Language', '-1', '0', '1'])
    fig = go.Figure(data=[
    go.Bar(name='0', x=df['Language'], y=df['0']),
    go.Bar(name='1', x=df['Language'], y=df['1']),
    go.Bar(name='NaN', x=df['Language'], y=df['-1'])
    ])
    fig.update_layout(title=model)
    fig.show()

In [58]:
for model in glob.glob('preds/pred*prompt3'):
    fig=go.Figure()
    ds = Dataset.from_file(f"{model}/data-00000-of-00001.arrow")
    df_pandas = ds.to_pandas()
    complete_list = []
    for lang in config.languages:
        fig=go.Figure()
        group = df_pandas.groupby(f'{lang}: Pred label')[f'{lang}: Pred label'].count().to_frame('count').reset_index()

        missing = [x for x in [-1,1,0] if x not in group[f'{lang}: Pred label'].unique()]
        if len(missing) > 0:
            for i in range(len(missing)):
                group.at[group.index[-1]+1+i, f'{lang}: Pred label'] = missing[i]
                group.at[group.index[-1]+1+i, 'count'] = 0


        complete_list.append((lang, group[group[f'{lang}: Pred label'] == -1]['count'].iloc[0], group[group[f'{lang}: Pred label'] == 0]['count'].iloc[0], group[group[f'{lang}: Pred label'] == 1]['count'].iloc[0]))
    
    df = pd.DataFrame(complete_list, columns =['Language', '-1', '0', '1'])
    fig = go.Figure(data=[
    go.Bar(name='0', x=df['Language'], y=df['0']),
    go.Bar(name='1', x=df['Language'], y=df['1']),
    go.Bar(name='NaN', x=df['Language'], y=df['-1'])
    ])
    fig.update_layout(title=model)
    fig.show()

In [133]:
fig=go.Figure()
for lang in config.languages:
    total_frame = pd.DataFrame()
    i=1
    for prompt in glob.glob('preds/pred*Mistral*'):
        ds = Dataset.from_file(f"{prompt}/data-00000-of-00001.arrow")
        df_pandas = ds.to_pandas()
        df_pandas.loc[df_pandas['Bias Type']  == ' gender', 'Bias Type'] = 'gender'
        subset = df_pandas[['Index', 'Subset', 'Bias Type', 'gold',f'{lang}: Pred label']]
        subset[f'match prompt {str(i)}'] = subset[f'{lang}: Pred label'] == subset['gold']
        subset = subset.rename(columns={f'{lang}: Pred label': f'{lang}: Pred label prompt {str(i)}'})
        if len(total_frame) == 0:
            total_frame = subset
        else:
            total_frame = pd.merge(total_frame, subset, on=['Index', 'Subset','Bias Type', 'gold'])
        i+=1

    
    group = (total_frame.groupby(['gold', 'Bias Type'])['match prompt 1', 'match prompt 2', 'match prompt 3'].sum() / total_frame.groupby(['gold', 'Bias Type'])['match prompt 1', 'match prompt 2', 'match prompt 3'].count()) *100
    
    group['mean_acc'] = group[['match prompt 1','match prompt 2', 'match prompt 3']].mean(axis=1)

    group = group.reset_index()
    group = group[group['Bias Type'].isin(['gender', 'ethnicity', 'nationality', 'age', 'physical appearance', 'political'])]

    fig.add_trace(go.Bar(x=group['Bias Type'], y=group['mean_acc'], name=lang))



fig.update_layout(title='Mistral')
fig.show()

/var/folders/8g/lh2lrvsj7837bypfsr65hcb80000gn/T/ipykernel_70531/3490465145.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/8g/lh2lrvsj7837bypfsr65hcb80000gn/T/ipykernel_70531/3490465145.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/8g/lh2lrvsj7837bypfsr65hcb80000gn/T/ipykernel_70531/3490465145.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [134]:
fig=go.Figure()
for lang in config.languages:
    total_frame = pd.DataFrame()
    i=1
    for prompt in glob.glob('preds/pred*Llama*'):
        ds = Dataset.from_file(f"{prompt}/data-00000-of-00001.arrow")
        df_pandas = ds.to_pandas()
        df_pandas.loc[df_pandas['Bias Type']  == ' gender', 'Bias Type'] = 'gender'
        subset = df_pandas[['Index', 'Subset', 'Bias Type', 'gold',f'{lang}: Pred label']]
        subset[f'match prompt {str(i)}'] = subset[f'{lang}: Pred label'] == subset['gold']
        subset = subset.rename(columns={f'{lang}: Pred label': f'{lang}: Pred label prompt {str(i)}'})
        if len(total_frame) == 0:
            total_frame = subset
        else:
            total_frame = pd.merge(total_frame, subset, on=['Index', 'Subset','Bias Type', 'gold'])
        i+=1

    group = (total_frame.groupby(['gold', 'Bias Type'])['match prompt 1', 'match prompt 2', 'match prompt 3'].sum() / total_frame.groupby(['gold', 'Bias Type'])['match prompt 1', 'match prompt 2', 'match prompt 3'].count()) *100

    group['mean_acc'] = group[['match prompt 1','match prompt 2', 'match prompt 3']].mean(axis=1)

    group = group.reset_index()
    group = group[group['Bias Type'].isin(['gender', 'ethnicity', 'nationality', 'age', 'physical appearance', 'political'])]

    fig.add_trace(go.Bar(x=group['Bias Type'], y=group['mean_acc'], name=lang))



fig.update_layout(title='Llama3')
fig.show()

/var/folders/8g/lh2lrvsj7837bypfsr65hcb80000gn/T/ipykernel_70531/2037345545.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/8g/lh2lrvsj7837bypfsr65hcb80000gn/T/ipykernel_70531/2037345545.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/8g/lh2lrvsj7837bypfsr65hcb80000gn/T/ipykernel_70531/2037345545.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [135]:
fig=go.Figure()
for lang in config.languages:
    total_frame = pd.DataFrame()
    i=1
    for prompt in glob.glob('preds/pred*Qwen*'):
        ds = Dataset.from_file(f"{prompt}/data-00000-of-00001.arrow")
        df_pandas = ds.to_pandas()
        df_pandas.loc[df_pandas['Bias Type']  == ' gender', 'Bias Type'] = 'gender'
        subset = df_pandas[['Index', 'Subset', 'Bias Type', 'gold',f'{lang}: Pred label']]
        subset[f'match prompt {str(i)}'] = subset[f'{lang}: Pred label'] == subset['gold']
        subset = subset.rename(columns={f'{lang}: Pred label': f'{lang}: Pred label prompt {str(i)}'})
        if len(total_frame) == 0:
            total_frame = subset
        else:
            total_frame = pd.merge(total_frame, subset, on=['Index', 'Subset','Bias Type', 'gold'])
        i+=1

    group = (total_frame.groupby(['gold', 'Bias Type'])['match prompt 1', 'match prompt 2', 'match prompt 3'].sum() / total_frame.groupby(['gold', 'Bias Type'])['match prompt 1', 'match prompt 2', 'match prompt 3'].count()) *100

    group['mean_acc'] = group[['match prompt 1','match prompt 2', 'match prompt 3']].mean(axis=1)

    group = group.reset_index()
    group = group[group['Bias Type'].isin(['gender', 'ethnicity', 'nationality', 'age', 'physical appearance', 'political'])]

    fig.add_trace(go.Bar(x=group['Bias Type'], y=group['mean_acc'], name=lang))



fig.update_layout(title='Qwen2')
fig.show()

/var/folders/8g/lh2lrvsj7837bypfsr65hcb80000gn/T/ipykernel_70531/917880078.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/8g/lh2lrvsj7837bypfsr65hcb80000gn/T/ipykernel_70531/917880078.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/8g/lh2lrvsj7837bypfsr65hcb80000gn/T/ipykernel_70531/917880078.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [136]:
fig=go.Figure()
for lang in config.languages:
    total_frame = pd.DataFrame()
    i=1
    for prompt in glob.glob('preds/pred*bloom*'):
        ds = Dataset.from_file(f"{prompt}/data-00000-of-00001.arrow")
        df_pandas = ds.to_pandas()
        df_pandas.loc[df_pandas['Bias Type']  == ' gender', 'Bias Type'] = 'gender'
        subset = df_pandas[['Index', 'Subset', 'Bias Type', 'gold',f'{lang}: Pred label']]
        subset[f'match prompt {str(i)}'] = subset[f'{lang}: Pred label'] == subset['gold']
        subset = subset.rename(columns={f'{lang}: Pred label': f'{lang}: Pred label prompt {str(i)}'})
        if len(total_frame) == 0:
            total_frame = subset
        else:
            total_frame = pd.merge(total_frame, subset, on=['Index', 'Subset','Bias Type', 'gold'])
        i+=1

    group = (total_frame.groupby(['gold', 'Bias Type'])['match prompt 1', 'match prompt 2', 'match prompt 3'].sum() / total_frame.groupby(['gold', 'Bias Type'])['match prompt 1', 'match prompt 2', 'match prompt 3'].count()) *100

    group['mean_acc'] = group[['match prompt 1','match prompt 2', 'match prompt 3']].mean(axis=1)

    group = group.reset_index()
    group = group[group['Bias Type'].isin(['gender', 'ethnicity', 'nationality', 'age', 'physical appearance', 'political'])]

    fig.add_trace(go.Bar(x=group['Bias Type'], y=group['mean_acc'], name=lang))



fig.update_layout(title='Bloomz')
fig.show()

/var/folders/8g/lh2lrvsj7837bypfsr65hcb80000gn/T/ipykernel_70531/4227007230.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/8g/lh2lrvsj7837bypfsr65hcb80000gn/T/ipykernel_70531/4227007230.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/8g/lh2lrvsj7837bypfsr65hcb80000gn/T/ipykernel_70531/4227007230.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th